In [329]:
import pandas as pd
import time
import requests
import re
from bs4 import BeautifulSoup as bs
from datetime import datetime
from dateutil import parser
from dateutil.parser import ParserError

def Get_BS(webpage):
    
    response = requests.get(webpage)
    
    if response.status_code == 200:
        anime_webpage = bs(response.text, 'html.parser')
        print(f'{webpage} scrape successful')
    else:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}")
    
    return anime_webpage
    
def IndexPage(_):
    
    return {element.find("span").get_text(): idx for idx, element in enumerate(_) if element.find("span")}

def GenerateDictionary(alist,aindex,atitle):
    
    AnimeDict = {}  
    
    AnimeDict["Title"] = atitle
    
    i = aindex.get("English:")
    AnimeDict["English_Title"] = alist[i].find("span").next_sibling.strip() if i is not None else None
    
    i = aindex.get("Episodes:")
    AnimeDict["Episodes"] = alist[i].find("span").next_sibling.strip() if i is not None else None
    if AnimeDict["Episodes"] == 'Unknown':
        AnimeDict["Episodes"] = None
    else:
        AnimeDict["Episodes"] = float(AnimeDict["Episodes"])
    
    i = aindex.get("Premiered:")
    AnimeDict["Season"] = alist[i].find("a").get_text() if i is not None else None

    i = aindex.get("Aired:")
    try:
        range = alist[i].find("span").next_sibling.strip().split(' to ')
    except AttributeError: 
        start_date = None
        end_date = None
    else:
        AnimeDict["Start_Date"] = DateParser(range[0])
        AnimeDict["End_Date"] = DateParser(range[1])    
    
    i = aindex.get("Studios:")
    AnimeDict["Studios"] = [_.get_text() for _ in alist[i].find_all("a")] if i is not None else None
    if AnimeDict["Studios"] == ['add some']:
        AnimeDict["Studios"] = None 
        
    i = aindex.get("Source:")
    AnimeDict["Source"] = alist[i].find("span").next_sibling.strip() if i is not None else None

    i = next((key for key in aindex if re.search(r'Genre[s]?', key)), None)
    AnimeDict["Genres"] = [_.get_text() for _ in alist[aindex[i]].find_all("span",attrs={"itemprop":"genre"})] if i is not None else None

    i = next((key for key in aindex if re.search(r'Theme[s]?', key)), None)
    AnimeDict["Themes"] = [_.get_text() for _ in alist[aindex[i]].find_all("span", attrs={"itemprop": "genre"})] if i else None

    i = aindex.get("Demographic:")
    AnimeDict["Demographic"] = alist[i].find("span",attrs={"itemprop":"genre"}).get_text() if i is not None else None

    i = aindex.get("Rating:")
    AnimeDict["Rating"] = alist[i].find("span").next_sibling.strip() if i is not None else None

    i = aindex.get("Score:")
    try:
        AnimeDict["Score"] = float(alist[i].find("span",attrs={"itemprop":"ratingValue"}).get_text()) if i is not None else None
    except AttributeError:
        AnimeDict["Score"] = None
    
    i = aindex.get("Score:")
    try:
        AnimeDict["Users"] = int(alist[i].find("span",attrs={"itemprop":"ratingCount"}).get_text()) if i is not None else None
    except AttributeError: 
        AnimeDict["Users"] = None
    
    i = aindex.get("Members:")
    AnimeDict["Members"] = int(alist[i].find("span").next_sibling.strip().replace(',','')) if i is not None else None
    
    AnimeDict["Insert_Time"] = datetime.now()
    
    return AnimeDict

def DateParser(date):
    try:
        return parser.parse(date)
    except ParserError:
        return None

In [367]:
def GetAnimeList(link):
    season_webpage = Get_BS(link)
    season_anime = season_webpage.find("div", attrs={"class":"seasonal-anime-list js-seasonal-anime-list js-seasonal-anime-list-key-1"}).find_all("a", attrs={"class":"link-title"})
    season_anime = [anime['href'] for anime in season_anime]
    return season_anime

seasons = ['Fall', 'Summer', 'Spring', 'Winter']
years = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

anime_links = {}
for season in seasons: 
    for year in years: 
        key = f'{season}{year}'
        anime_links[key] = GetAnimeList(f'https://myanimelist.net/anime/season/{year}/{season}')
                
anime_links

https://myanimelist.net/anime/season/2014/Fall scrape successful
https://myanimelist.net/anime/season/2015/Fall scrape successful
https://myanimelist.net/anime/season/2016/Fall scrape successful
https://myanimelist.net/anime/season/2017/Fall scrape successful
https://myanimelist.net/anime/season/2018/Fall scrape successful
https://myanimelist.net/anime/season/2019/Fall scrape successful
https://myanimelist.net/anime/season/2020/Fall scrape successful
https://myanimelist.net/anime/season/2021/Fall scrape successful
https://myanimelist.net/anime/season/2022/Fall scrape successful
https://myanimelist.net/anime/season/2023/Fall scrape successful
https://myanimelist.net/anime/season/2014/Summer scrape successful
https://myanimelist.net/anime/season/2015/Summer scrape successful
https://myanimelist.net/anime/season/2016/Summer scrape successful
https://myanimelist.net/anime/season/2017/Summer scrape successful
https://myanimelist.net/anime/season/2018/Summer scrape successful
https://myanime

{'Fall2014': ['https://myanimelist.net/anime/52991/Sousou_no_Frieren',
  'https://myanimelist.net/anime/40357/Tate_no_Yuusha_no_Nariagari_Season_3',
  'https://myanimelist.net/anime/53887/Spy_x_Family_Season_2',
  'https://myanimelist.net/anime/47160/Goblin_Slayer_II',
  'https://myanimelist.net/anime/54595/Kage_no_Jitsuryokusha_ni_Naritakute_2nd_Season',
  'https://myanimelist.net/anime/55644/Dr_Stone__New_World_Part_2',
  'https://myanimelist.net/anime/54492/Kusuriya_no_Hitorigoto',
  'https://myanimelist.net/anime/52741/Undead_Unluck',
  'https://myanimelist.net/anime/52347/Shangri-La_Frontier__Kusoge_Hunter_Kamige_ni_Idoman_to_su',
  'https://myanimelist.net/anime/54918/Tokyo_Revengers__Tenjiku-hen',
  'https://myanimelist.net/anime/54362/Hametsu_no_Oukoku',
  'https://myanimelist.net/anime/54714/Kimi_no_Koto_ga_Daidaidaidaidaisuki_na_100-nin_no_Kanojo',
  'https://myanimelist.net/anime/52990/Keikenzumi_na_Kimi_to_Keiken_Zero_na_Ore_ga_Otsukiai_suru_Hanashi',
  'https://myanimelist

In [376]:
def GetAnimeDF(anime_links):
    
    Anime_List = []

    for i, link in enumerate(anime_links): 
        anime_webpage = Get_BS(link)
        print(f'{i + 1} out of {len(anime_links)} completed, waiting 5 seconds')
        time.sleep(5)
        title = anime_webpage.h1.get_text()
        info = anime_webpage.find_all("div", attrs={"class": "spaceit_pad"})

        index = IndexPage(info)
        Anime_List.append(GenerateDictionary(info, index, title))

    return pd.DataFrame(Anime_List)

In [351]:
SampleList = ['https://myanimelist.net/anime/52991/Sousou_no_Frieren', 'https://myanimelist.net/anime/40357/Tate_no_Yuusha_no_Nariagari_Season_3']

In [354]:
GetAnimeDF(SampleList)

https://myanimelist.net/anime/52991/Sousou_no_Frieren scrape successful
1 out of 2 completed
https://myanimelist.net/anime/40357/Tate_no_Yuusha_no_Nariagari_Season_3 scrape successful
2 out of 2 completed


,Title,English_Title,Episodes,Season,Start_Date,End_Date,Studios,Source,Genres,Themes,Demographic,Rating,Score,Users,Members,Insert_Time
0,Sousou no Frieren,Frieren: Beyond Journey's End,28.0,Fall 2023,2023-09-29,2024-03-17,[Madhouse],Manga,"[Adventure, Drama, Fantasy]",None,Shounen,PG-13 - Teens 13 or older,9.13,104178,376438,2023-12-17 09:52:07.677126
1,Tate no Yuusha no Nariagari Season 3,The Rising of the Shield Hero Season 3,12.0,Fall 2023,2023-10-06,2023-12-22,[Kinema Citrus],Light novel,"[Action, Adventure, Drama, Fantasy]",[Isekai],None,PG-13 - Teens 13 or older,7.34,24494,376280,2023-12-17 09:52:10.758061


In [355]:
season_links

['https://myanimelist.net/anime/season/2014/Fall',
 'https://myanimelist.net/anime/season/2015/Fall',
 'https://myanimelist.net/anime/season/2016/Fall',
 'https://myanimelist.net/anime/season/2017/Fall',
 'https://myanimelist.net/anime/season/2018/Fall',
 'https://myanimelist.net/anime/season/2019/Fall',
 'https://myanimelist.net/anime/season/2020/Fall',
 'https://myanimelist.net/anime/season/2021/Fall',
 'https://myanimelist.net/anime/season/2022/Fall',
 'https://myanimelist.net/anime/season/2023/Fall',
 'https://myanimelist.net/anime/season/2014/Summer',
 'https://myanimelist.net/anime/season/2015/Summer',
 'https://myanimelist.net/anime/season/2016/Summer',
 'https://myanimelist.net/anime/season/2017/Summer',
 'https://myanimelist.net/anime/season/2018/Summer',
 'https://myanimelist.net/anime/season/2019/Summer',
 'https://myanimelist.net/anime/season/2020/Summer',
 'https://myanimelist.net/anime/season/2021/Summer',
 'https://myanimelist.net/anime/season/2022/Summer',
 'https://mya

In [373]:
Anime2023 = anime_links['Winter2023'] + anime_links['Spring2023'] + anime_links['Summer2023'] + anime_links['Fall2023']
Anime2023

['https://myanimelist.net/anime/52991/Sousou_no_Frieren',
 'https://myanimelist.net/anime/40357/Tate_no_Yuusha_no_Nariagari_Season_3',
 'https://myanimelist.net/anime/53887/Spy_x_Family_Season_2',
 'https://myanimelist.net/anime/47160/Goblin_Slayer_II',
 'https://myanimelist.net/anime/54595/Kage_no_Jitsuryokusha_ni_Naritakute_2nd_Season',
 'https://myanimelist.net/anime/55644/Dr_Stone__New_World_Part_2',
 'https://myanimelist.net/anime/54492/Kusuriya_no_Hitorigoto',
 'https://myanimelist.net/anime/52741/Undead_Unluck',
 'https://myanimelist.net/anime/52347/Shangri-La_Frontier__Kusoge_Hunter_Kamige_ni_Idoman_to_su',
 'https://myanimelist.net/anime/54918/Tokyo_Revengers__Tenjiku-hen',
 'https://myanimelist.net/anime/54362/Hametsu_no_Oukoku',
 'https://myanimelist.net/anime/54714/Kimi_no_Koto_ga_Daidaidaidaidaisuki_na_100-nin_no_Kanojo',
 'https://myanimelist.net/anime/52990/Keikenzumi_na_Kimi_to_Keiken_Zero_na_Ore_ga_Otsukiai_suru_Hanashi',
 'https://myanimelist.net/anime/51297/Ragna_Cri

In [377]:
A23DF = GetAnimeDF(Anime2023)

https://myanimelist.net/anime/52991/Sousou_no_Frieren scrape successful
1 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/40357/Tate_no_Yuusha_no_Nariagari_Season_3 scrape successful
2 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/53887/Spy_x_Family_Season_2 scrape successful
3 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/47160/Goblin_Slayer_II scrape successful
4 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/54595/Kage_no_Jitsuryokusha_ni_Naritakute_2nd_Season scrape successful
5 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/55644/Dr_Stone__New_World_Part_2 scrape successful
6 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/54492/Kusuriya_no_Hitorigoto scrape successful
7 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/52741/Undead_Unluck scrape successful
8 out of 324 completed, waiting 5 seconds
https://myanimelist.net/ani

https://myanimelist.net/anime/56997/Sumikko_Gurashi__Sorairo_no_Mainichi scrape successful
67 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/56504/Chickip_Dancers_3rd_Season scrape successful
68 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/56677/Saekomdalkom_Catch_Tiniping scrape successful
69 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/50995/Zenryoku_Usagi_2023 scrape successful
70 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/56485/Shasha_and_Milo scrape successful
71 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/56484/Cabeon scrape successful
72 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/56986/Riseman scrape successful
73 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/57077/Totonoe_Sakuma-kun_by__sauna scrape successful
74 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/57132/Hero_Inside scra

https://myanimelist.net/anime/53526/Uma_Musume__Pretty_Derby_Season_3 scrape successful
132 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/53633/Bullbuster scrape successful
133 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/54803/Captain_Tsubasa_Season_2__Junior_Youth-hen scrape successful
134 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/42385/The_iDOLMSTER_Million_Live scrape successful
135 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/54858/Hypnosis_Mic__Division_Rap_Battle_-_Rhyme_Anima__ scrape successful
136 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/54716/Kibou_no_Chikara__Otona_Precure_23 scrape successful
137 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/44583/Rail_Romanesque_2 scrape successful
138 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/54638/Kawagoe_Boys_Sing scrape successful
139 out of 324 completed

https://myanimelist.net/anime/54431/Toaru_Ossan_no_VRMMO_Katsudouki scrape successful
198 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/50586/Migi_to_Dali scrape successful
199 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/53262/Hoshikuzu_Telepath scrape successful
200 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/52985/Dekoboko_Majo_no_Oyako_Jijou scrape successful
201 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/54041/16bit_Sensation__Another_Layer scrape successful
202 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/55153/Yuzuki-san_Chi_no_Yonkyoudai scrape successful
203 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/53881/Arknights__Touin_Kiro scrape successful
204 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/55310/Atarashii_Joushi_wa_Do_Tennen scrape successful
205 out of 324 completed, waiting 5 seconds
https://mya

https://myanimelist.net/anime/54103/Hikikomari_Kyuuketsuki_no_Monmon scrape successful
264 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/53833/Watashi_no_Oshi_wa_Akuyaku_Reijou scrape successful
265 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/52934/Konyaku_Haki_sareta_Reijou_wo_Hirotta_Ore_ga_Ikenai_Koto_wo_Oshiekomu scrape successful
266 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/55742/Mahoutsukai_no_Yome_Season_2_Part_2 scrape successful
267 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/54852/Kikansha_no_Mahou_wa_Tokubetsu_desu scrape successful
268 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/51215/Seijo_no_Maryoku_wa_Bannou_desu_Season_2 scrape successful
269 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/53300/Ojou_to_Banken-kun scrape successful
270 out of 324 completed, waiting 5 seconds
https://myanimelist.net/anime/53494/Boukensha

In [378]:
A23DF

,Title,English_Title,Episodes,Season,Start_Date,End_Date,Studios,Source,Genres,Themes,Demographic,Rating,Score,Users,Members,Insert_Time
0,Sousou no Frieren,Frieren: Beyond Journey's End,28.0,Fall 2023,2023-09-29,2024-03-17,[Madhouse],Manga,"[Adventure, Drama, Fantasy]",None,Shounen,PG-13 - Teens 13 or older,9.13,104202.0,376551,2023-12-17 10:20:44.909243
1,Tate no Yuusha no Nariagari Season 3,The Rising of the Shield Hero Season 3,12.0,Fall 2023,2023-10-06,2023-12-22,[Kinema Citrus],Light novel,"[Action, Adventure, Drama, Fantasy]",[Isekai],None,PG-13 - Teens 13 or older,7.34,24494.0,376287,2023-12-17 10:20:51.456680
2,Spy x Family Season 2,None,12.0,Fall 2023,2023-10-07,2023-12-23,"[Wit Studio, CloverWorks]",Manga,"[Action, Comedy]",[Childcare],Shounen,PG-13 - Teens 13 or older,8.20,47323.0,360609,2023-12-17 10:20:58.220773
3,Goblin Slayer II,None,12.0,Fall 2023,2023-10-06,2023-12-22,[LIDENFILMS],Light novel,"[Action, Adventure, Fantasy]",[Gore],None,R - 17+ (violence & profanity),7.42,25471.0,284459,2023-12-17 10:21:05.383457
4,Kage no Jitsuryokusha ni Naritakute! 2nd Season,The Eminence in Shadow Season 2,12.0,Fall 2023,2023-10-04,2023-12-20,[Nexus],Light novel,"[Action, Comedy, Fantasy]","[Isekai, Reincarnation]",None,R - 17+ (violence & profanity),8.55,47172.0,251595,2023-12-17 10:21:12.238994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,Choeganggyeongchal Miniforce,None,NaN,Fall 2023,2023-11-28,NaT,[SAMG Entertainment ],Original,[Action],"[Anthropomorphic, Mecha]",Kids,PG - Children,NaN,NaN,80,2023-12-17 10:54:05.122972
320,Hello Carbot Season 14,None,NaN,Fall 2023,2023-11-05,NaT,None,Original,[Action],"[Mecha, Racing]",Kids,PG - Children,NaN,NaN,66,2023-12-17 10:54:10.655763
321,Sinbi Apateu: Ghost Ball Zero - Du Beonjjae Iyagi,None,NaN,Fall 2023,2023-11-30,NaT,None,Original,"[Fantasy, Supernatural]",None,Kids,G - All Ages,NaN,NaN,66,2023-12-17 10:54:16.609806
322,Geomeongsupeul Jikyeola! Birdy Friends,None,26.0,Fall 2023,2023-11-23,2024-05-16,None,Original,[Adventure],None,Kids,PG - Children,NaN,NaN,56,2023-12-17 10:54:22.222773
